In [56]:
import re
from collections import Counter
from helpers import data
import doctest

doctest.testmod()

TestResults(failed=0, attempted=16)

In [8]:
images_arr = data(20, parser=lambda l: l.split("\n"), sep="\n\n")
images = {}
for first, *rest in images_arr:
    image_id = re.search("\d+", first).group()
    images[int(image_id)] = rest

In [9]:
len(images)

144

In [10]:
images[3301]

['##..#...#.',
 '.....#...#',
 '#...#...##',
 '#....#...#',
 '#....#.#..',
 '.#..#.#.##',
 '.........#',
 '##...#....',
 '#...##...#',
 '#..#..#.##']

**Part 1:** Find the correct order and orientation of images so that their borders line up. Multiply the IDs of the four corner images. 

In order to avoid having to define a grid, I'll find a corner tile and set it to
the top-left (so maybe rotate it a bit). Then I go left-to-right and find matches
to the edges, then top-to-down.

In [23]:
def transpose(grid):
    # Extra list calls are to get a list of lists instead of list of tuples
    return list(map(list, zip(*grid)))


def flip_horizontally(grid):
    flipped_grid = []
    for row in grid:
        flipped_grid.append(list(reversed(row)))
    return flipped_grid

In [12]:
# Get top, bottom, left, right of a 2D array
def top(grid):
    """
    >>> top([['A', 'B'], ['C', 'D']])
    ['A', 'B']
    """
    return grid[0]


def bottom(grid):
    """
    >>> bottom([['A', 'B'], ['C', 'D']])
    ['C', 'D']
    """
    return grid[-1]


def left(grid):
    """
    >>> left([['A', 'B'], ['C', 'D']])
    ['A', 'C']
    """
    return top(transpose(grid))


def right(grid):
    """
    >>> right([['A', 'B'], ['C', 'D']])
    ['B', 'D']
    """
    return bottom(transpose(grid))

In [24]:
def rotate_clockwise(grid):
    """
    >>> rotate_clockwise([['A', 'B'], ['C', 'D']])
    [['C', 'A'], ['D', 'B']]
    """
    # Transpose and flip vertical
    return flip_horizontally(transpose(grid))

In [63]:
def fit_left(side, grid):
    """
    Check if any rotated or flipped version of `grid` has a left side that
    matches `side`. Return that orientation if it exists, else None.

    >>> fit_left(['A', 'B'], [['A', 'B'], ['C', 'D']])
    [['A', 'C'], ['B', 'D']]
    >>> fit_left(['A', 'B'], [['A', 'C'], ['D', 'B']])
    >>> m = [['A', 'B', 'C'], ['D', 'E', 'F'], ['G', 'H', 'I']]
    >>> fit_left(['A', 'D', 'G'], m) is not None
    True
    >>> fit_left(['G', 'H', 'I'], m) is not None
    True
    >>> fit_left(['I', 'F', 'C'], m) is not None
    True
    >>> fit_left(['C', 'B', 'A'], m) is not None
    True
    >>> fit_left(['G', 'D', 'A'], m) is not None
    True
    >>> fit_left(['I', 'H', 'G'], m) is not None
    True
    >>> fit_left(['C', 'F', 'I'], m) is not None
    True
    >>> fit_left(['A', 'B', 'C'], m) is not None
    True
    """
    for _ in range(4):
        if left(grid) == side:
            return grid
        grid = rotate_clockwise(grid)

    grid = transpose(grid)
    for _ in range(4):
        if left(grid) == side:
            return grid
        grid = rotate_clockwise(grid)

    return None

In [69]:
# Just find corner tiles without assembling image
corner_tiles = []
for image_id, image in images.items():
    matched_sides = 0

    # Check if each side is found at least once
    for i, side in enumerate((top(image), bottom(image), left(image), right(image))):
        for check_image_id, check_image in images.items():
            if image_id == check_image_id:
                continue
            if fit_left(side, check_image):
                matched_sides += 1
                break
    if matched_sides == 2:
        corner_tiles.append(image)

In [70]:
# Broken somewhere
len(corner_tiles)

123

In [ ]:
# HINT: Choose an arbitrary orientation for all edges instead of worrying about
# flipping. And use 1D arrays, so lists of strings instead of 2D arrays. Cleaner.